In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# -*- coding: utf-8 -*-
"""tinygptforcollab

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/19DiOjHtL-UEa69WlOcMsmRnkrNc5NmaS
"""

# -*- coding: utf-8 -*-
"""tinygpt.ipynb
Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/109yvdricQdl9D43KPCeh829Y89TGUcvC


"""

import torch
device='cuda' if torch.cuda.is_available() else 'cpu'
eval_iters=200
n_embd=384
lr=3e-4
max_iters=5000
batch_size=64
block_size=256
num_heads=6
eval_inter=500
dropout=0.2
nlayerb=6

chars=['\n', ' ', '!', '"', '#', '$', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '\xad', '´', 'é', 'ñ', '\u200a', '\u200b', '–', '—', '‘', '’', '“', '”', '…', '🎓']
vocab_size=len(chars)

itos={i:s for i, s in enumerate(chars)}
decode = lambda d: "".join([itos[c] for c in d])








import torch
from torch import nn as nn
import torch.nn.functional as F

class Head(nn.Module):
   def __init__(self, head_size):
    super().__init__()
    self.key=nn.Linear(n_embd, head_size, bias=False)
    self.query=nn.Linear(n_embd, head_size, bias=False)
    self.value=nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.ddp=nn.Dropout(dropout)
   def forward(self, x):
      B, T, C = x.shape
      k=self.key(x)
      q=self.query(x)
      v=self.value(x)
      wei=q@k.transpose(-2, -1) * k.shape[-1]**-0.5
      wei=wei.masked_fill(self.tril[:T, :T]==0, float('-inf'))
      wei=F.softmax(wei, dim=-1)
      wei=self.ddp(wei)
      if torch.isnan(wei).any() or torch.isinf(wei).any():
          print("NaN or Inf values detected in the probability tensor.")
    # Add debugging information or further investigation her
      else:
         out=wei@v
         return out

class MultiHeadAttention(nn.Module):
   def __init__(self, num_heads, head_size):
      super().__init__()
      self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
      self.proj=nn.Linear(n_embd, n_embd)
      self.ddp=nn.Dropout(dropout)
   def forward(self, x):
      out=torch.cat([h(x) for h in self.heads], dim=-1)
      out=self.ddp(self.proj(out))
      return out


class feedfoward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.net=nn.Sequential(nn.Linear(n_embd, 4*n_embd), nn.ReLU(), nn.Linear(4*n_embd, n_embd), nn.Dropout(dropout))
  def forward(self, x):
    ffd=self.net(x)
    return ffd

class Block(nn.Module):
  def __init__(self, n_embd, num_heads):
    super().__init__()
    head_size=n_embd//num_heads
    self.heads=MultiHeadAttention(num_heads, head_size)
    self.ffd=feedfoward(n_embd)
    self.lln=nn.LayerNorm(n_embd)
  def forward(self, x):
    x=x+self.heads(self.lln(x))
    x=x+self.ffd(self.lln(x))
    return x







torch.manual_seed(1337)
class BigramModel(nn.Module):
    def __init__ (self):
        super().__init__()
        self.tokenembeddingtable=nn.Embedding(vocab_size, n_embd)
        self.positionembeddingtable=nn.Embedding(block_size, n_embd)
        self.heads=MultiHeadAttention(num_heads, n_embd//4)
        self.ffd=feedfoward(n_embd)
        self.block=nn.Sequential(*[Block(n_embd, num_heads=num_heads)for _ in range(nlayerb)])
        self.lnf=nn.LayerNorm(n_embd)
        self.lmhead=nn.Linear(n_embd, vocab_size)
    def forward(self, idx, targets=None):
        idx = idx.to(device)
        B, T= idx.shape
        tok_emb=self.tokenembeddingtable(idx)
        pos_emb=self.positionembeddingtable(torch.arange(T, device=device))
        x=tok_emb+pos_emb
        x=self.block(x)
        x=self.lnf(x)

        logits=self.lmhead(x)


        B, T, C =logits.shape

        if targets is not None:
          logits=logits.view(B*T, C)
          targets=targets.to(device)

          targets=targets.view(B*T)
          loss=F.cross_entropy(logits, targets)
        else:
          loss=None
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
          idx_cong=idx[:, -block_size:]
          logits, loss=self(idx_cong)
          logits=logits[:, -1, :]
          prob=F.softmax(logits, dim=-1)
          idx_next=torch.multinomial(prob, num_samples=1)
          idx=torch.cat((idx, idx_next), dim=1)


        return idx













bm1=BigramModel()
bm1=bm1.to(device)
bm1.eval()
bm1.load_state_dict(torch.load('/content/drive/MyDrive/tinygpttinystories.pt'))







<All keys matched successfully>

In [5]:
n_of_tokens=1000 #no. of tokens to generate
print(decode(bm1.generate(torch.zeros((1, 1), dtype=torch.long, device=device), n_of_tokens)[0].tolist()))


Once upon a time, there was a small monster, who always cheered there. Everyone was heavy that the monster would come to the park, but sometimes they are to listen.
Lily was amazed. She told about the park every day, they both took them to the green stick and went to Cheep all day. The day, the brother was very over. And they still played together every day, thanks they still pry and their brother, and they ate their daughter. Billy was a grown-up three years old and put them in his heart.
<|endoftext|>

Once, there was a little girl named Emma who was cold and good after her. She was always very cold. One day, Amy hopped. Her mom put a big bag and mean in. She realized that sharing mark with her.
But some was not not open to protect her. She also slipped and grass. The little girl would not play. She figured out that she was cool. She was happy against the little girl and then she saw a kind lady on her face.
<|endoftext|>

John and Amy love to play each other. They like playing with